# Initial Setup

## Libraries

In [1]:
import json
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from PyPDF2 import PdfReader
import gradio as gr
import agent_tools

/Users/goldenmeta/Github/Career-Agent/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Setups

In [2]:
# OpenAI Setup
load_dotenv(override=True)
openai = OpenAI()

# Pushover Setup
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [3]:
# The function will push message in the form of a payload to the phone based on predetermined parameters. 
def push_message(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

# Tools and Handler

## Tools
* Tool 1: Which will handle collecting the user's information in the case they would like to contact 'Kailas' or the pdf profile user.
* Tool 2: Which will handle collecting an questions which was or cannot be answered by the LLM. The tool will send a ping notification to my phone.

In [4]:
# function if whether the user would like to be in touch with us
def record_user_details(email, name="name not provided", notes="notes not provided"):
    # if provided with only email was provided
    if name == "name not provided" and notes == "notes not provided":
        push_message(f'{email} has shown interest in connecting with you!')
    # If the email and name was provided, but not notes. 
    elif name != "name not provided" and notes == "notes not provided":
        push_message(f'{name} has shown interest in connecting with you!\nEmail: {email}')
    # if the email and notes are provided, but not the name. 
    elif name == "name not provided" and notes != "notes not provided":
        push_message(f'{email} has shown interest in connecting with you!\nNotes: {notes}')
    else:
        push_message(f"{name} has show interest in connecting with you!\nEmail: {email}\nNotes: {notes}")
    return {"Recorded:": "ok"}

## Handler

In [5]:
def handle_tool_calls(tool_calls):
    '''
    A function which takes in a list of tool calls, and executes them (sends a notification to phone) based on their prompt.
    Furthermore, each tool is already pre-mapped to a function, all this function does it search and execute like a 'dumb' agent. 
    '''
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

# Development

## PDF-Reader

In [6]:
'''
MacOS
'''
linkedin_pdf_path = r"/Users/goldenmeta/Github/Career-Agent/3_resources/Linkedln.pdf"
linkedin_website_path = r"/Users/goldenmeta/Github/Career-Agent/3_resources/LinkedlnLink.txt"
resume_pdf_path = r"/Users/goldenmeta/Github/Career-Agent/3_resources/Resume.pdf"

In [7]:
def pdf_reader(pdf: PdfReader) -> str:
    '''
    This function will return a string based lining of all the contents in the pdf. 
    This will typicallly be used for the pdf linkedin and the resume. 
    '''
    pdf_text = ""
    for page in pdf.pages:
        text = page.extract_text()
        if text:
            pdf_text += text
    return pdf_text

In [8]:
# This can be your own name (replace with with Kailas Thonnithodi with your own name)
full_name = "Kailas Thonnithodi" 

linkedin_pdf_reader = PdfReader(linkedin_pdf_path)
resume_pdf_reader = PdfReader(resume_pdf_path)

linkedin = pdf_reader(linkedin_pdf_reader)
resume = pdf_reader(resume_pdf_reader)
linkedin_link = ""

# Reads the website link (i could paste the weblink here instead, however for sourcing purposes I will do it in a .txt file)
with open(linkedin_website_path, 'r') as link:
    linkedin_link = link.readline()

In [9]:
resume

'KAILAS THONNITHODI  \nPhone:  0435305756 , Email:  kthonnithodi@gmail.com  \nLinkedIn:  https://www.linkedin.com/in/kailas -thonnithodi/  \n \nCAREER PROFILE  \nFirst  year Masters  of Artificial Intelligence student with a passionate work ethic and desire to \nconsistently adapt and learn. Partaken in leading various computational and machine learning projects \nat Deakin University to deliver high -quality results. Looking for a graduate  role as an AI Engineer , \nMachine Learning Engineer or Data scientist.  \n \nEDUCATION  \n03/2025 –  Masters of Artificial Intelligence  \nMonash University, Clayton  \n• Commencement: March  2025  \n• Expected Completion: October 2027  \n• Average: High Distinction  \n \n03/2021  – 10/2024  Bachelor of Artificial Intelligence  \nDeakin University, Burwood  \n• Month Year Completed : Dec  2024  \n• Average: Distinction  \n \n01/2020 – 12/2020  Victorian Certification of Education  \nMount Waverley Secondary College, Mount Waverley  \n \nEXPERIENCE

## System Prompt

In [10]:
system_prompt = f"You are acting as {full_name}. You are answering questions on {full_name}'s website, \
    particularly questions related to {full_name}'s career, background, skills and experience. \
    Your responsibility is to represent {full_name} for interactions on the website as faithfully as possible. \
    Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
    The following are tools which you can called in certain scenarios. \
    If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using \'record_user_details\' tool. \
    I have given you all content regarding my resume and the linkedin profile. \
    Also I would like to emphasis that I want you to call only one tool at a time (if called, basically each time the user finishes their prompt)."

system_prompt += f"\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"\n\n## LinkedIn Link:\n{linkedin_link}\n\n"
system_prompt += f"\n\n## Resume:\n{resume}\n\n"

## Chat function

## Evaluator Model

In [11]:
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [12]:
# giving the evaluator an initalisatino state; the purpose of the evaluating another model's response. 
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {full_name} and is representing {full_name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {full_name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"\n\n## Resume:\n{resume}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback. Furthermore give it a score from 1-10."

In [13]:
# Giving the evaluator tool a user prompt. This will check how well the agent was able to give a response based on action. 
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Heres's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable."
    return user_prompt

In [14]:
evaluator_model = OpenAI(
    api_key=os.getenv("GEMINI_API_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta"
)

evaluator_model_name = "gemini-2.0-flash"
agent_model = "gpt-4o-mini"

In [15]:
# returns an evaluation of the model's response to the orignal user's sentence. 
def evaluate(reply, message, history) -> Evaluation:
    messages = [{"role": "system","content": evaluator_system_prompt}] + [{"role":"user","content": evaluator_user_prompt(reply, message, history)}]
    response = evaluator_model.chat.completions.parse(model=evaluator_model_name, messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [16]:
# creating a rerunning features which will create a new instance of the model which will then be used in asking the question again. 
# furthermore, the question which will be targetted will based on the score; if the score is below 1-5, this will lead to a less purpose solution.
# if the score is more than 5, this is considered acceptable, therefore continue, else re run, thinking and come up with a better solution for the user's question.
def rerun(reply, message, history, feedback):
    updated_sys_prompt = system_prompt + "\n\n## Previous answer rejected.\nYou just tried reply, however the quality control rejected your answer."
    updated_sys_prompt += f"## Your attempted answer:\n{reply}\n"
    updated_sys_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_sys_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=agent_model, messages=messages)
    return response.choices[0].message.content
           

## Running Model

In [17]:
# for the following agent, I will be using the gpt 4o mini model due to it's relative light weight and efficincy when producing a reasonable answer. 
tools = agent_tools.tools

def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    done = False
    while not done:
        # the response completed during the llms call
        response = openai.chat.completions.create(model=agent_model, messages=messages, tools=tools)
        reply = response.choices[0].message.content

        # Checking the evaluation based on the openai message board.
        evaluation = evaluate(reply, message, history)
        feedback = evaluation.feedback

        # If the message reasoning is not acceptable (which was parsed), then rerun the model to produce an acceptable response. 
        if evaluation.is_acceptable:
            # This is just for developers to see; making sure that the repsonse was actually gone through the evaluation too. 
            print(f"Pass evaluation - {feedback}")
        else:
            print(f"Failed evaluation - {feedback}")
            reply = rerun(reply, message, history, feedback)

        finish_reason = response.choices[0].finish_reason
        # If a tool is call, use the tool handler for committing.
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True

    return response.choices[0].message.content

## Tester using Gradio

In [18]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
